In [62]:
import sys
import os

# Ruta al directorio raíz del proyecto (ajústala si es necesario)
ruta_raiz = os.path.abspath(r"C:\Users\Germán Llorente\Desktop\germiprogramer\TFG-Agente-de-Scouting")

if ruta_raiz not in sys.path:
    sys.path.append(ruta_raiz)

In [63]:
import importlib
import funciones_analisis.estadisticas

importlib.reload(funciones_analisis.estadisticas)
importlib.reload(funciones_analisis.generales)

from funciones_analisis.estadisticas import *
from funciones_analisis.generales import *
from statsbombpy import sb
import pandas as pd
from mplsoccer import VerticalPitch, Pitch
from highlight_text import ax_text, fig_text
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.patheffects as path_effects
import seaborn as sns
import glob

In [64]:
# Cargar los datasets
jugadores_laliga = pd.read_csv('datos/datos_jugadores_v3/jugadores_laliga.csv', encoding="utf-8", skipinitialspace=True)
jugadores_premier = pd.read_csv('datos/datos_jugadores_v3/jugadores_premier.csv')
jugadores_seriea = pd.read_csv('datos/datos_jugadores_v3/jugadores_seriea.csv')
jugadores_bundesliga = pd.read_csv('datos/datos_jugadores_v3/jugadores_bundesliga.csv')

In [65]:
for i, df in enumerate([jugadores_laliga, jugadores_premier, jugadores_seriea, jugadores_bundesliga], start=1):
    print(f"\nDataset {i} columnas:")
    print(list(df.columns))



Dataset 1 columnas:
['player_id', 'player_name', 'team', 'team_id', 'posiciones', 'partidos_jugados', 'overall', 'potential', 'value_eur', 'wage_eur', 'age', 'dob', 'height_cm', 'weight_kg', 'club_jersey_number', 'club_loaned_from', 'club_contract_valid_until', 'nationality_id', 'nationality_name', 'preferred_foot', 'release_clause_eur', 'competition', 'tackle_success_rate', 'tackles_successful', 'interception_success_rate', 'interceptions', 'clearances', 'blocks', 'head_clearances', 'head_clearances_won', 'head_clearance_success_rate', 'headed_shots_total', 'headed_shots_after_duel', 'headed_shot_duel_rate', 'total_passes', 'completed_passes', 'incomplete_passes', 'passes_out', 'offside_passes', 'failed_passes', 'pass_completion_rate', 'avg_pass_length', 'ground_passes', 'low_passes', 'high_passes', 'ground_pass_percentage', 'low_pass_percentage', 'high_pass_percentage', 'crosses_total', 'crosses_completed', 'cutbacks_total', 'cutbacks_completed', 'switches_total', 'switches_complete

In [66]:
# Unir todos los datasets (uno debajo del otro)
jugadores_total = pd.concat([jugadores_laliga, jugadores_premier, jugadores_seriea, jugadores_bundesliga])
jugadores_total

,player_id,player_name,team,team_id,posiciones,partidos_jugados,overall,potential,value_eur,wage_eur,...,key_passes_per90,chances_created_per90,goals_conceded_per90,xg_total_per90,goals_minus_xg_per90,PSxG_per90,psxg_minus_goals_conceded_per90,yellow_cards_per90,red_cards_per90,posicion_mas_jugada
0,3023,Yuri Berchiche Izeta,Real Sociedad,210,['Left Back'],21,71,76,1800000.0,20000.0,...,0.598772,0.783009,0.000000,0.017631,-0.017503,0.000000,0.000000,0.460594,0.0,Left Back
1,3063,Danilo Luiz da Silva,Real Madrid,220,"['Right Back', 'Left Back', 'Right Wing']",24,80,86,16500000.0,90000.0,...,0.505618,0.716292,0.000000,0.040390,0.043820,0.000000,0.000000,0.252809,0.0,Right Back
2,3084,Christian Atsu Twasam,Málaga,223,"['Left Midfield', 'Right Midfield', 'Left Cent...",11,63,69,375000.0,4000.0,...,1.956522,1.956522,0.000000,0.410496,0.241304,0.000000,0.000000,0.000000,0.0,Right Midfield
3,3130,Gaël Kakuta,Sevilla,213,"['Left Midfield', 'Right Wing']",2,76,78,6500000.0,60000.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.967742,0.0,Left Midfield
4,3160,Rene Krhin,Granada,1049,"['Right Defensive Midfield', 'Left Defensive M...",23,75,80,4500000.0,35000.0,...,0.240642,0.240642,0.000000,0.031147,-0.031283,0.000000,0.000000,0.180481,0.0,Right Defensive Midfield
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423,400621,Tohouri Zahoui Constant Djakpa,Eintracht Frankfurt,184,"['Left Back', 'Left Wing Back', 'Left Midfield...",11,75,75,3100000.0,40000.0,...,0.851351,0.851351,0.000000,0.019767,-0.019459,0.000000,0.000000,0.243243,0.0,Left Back
424,400622,Joel Gerezgiher,Eintracht Frankfurt,184,['Left Midfield'],3,60,72,300000.0,2000.0,...,0.978261,0.978261,0.000000,0.304519,-0.303261,0.000000,0.000000,0.978261,0.0,Left Midfield
425,400627,Dario Kresic,Bayer Leverkusen,904,['Goalkeeper'],1,68,68,675000.0,10000.0,...,0.000000,0.000000,1.914894,0.000000,0.000000,1.780851,-0.134043,0.000000,0.0,Goalkeeper
426,400654,Roel Brouwers,Borussia Mönchengladbach,185,"['Right Center Back', 'Left Center Back', 'Rig...",8,76,76,2300000.0,50000.0,...,0.000000,0.000000,0.000000,0.025924,-0.025806,0.000000,0.000000,0.000000,0.0,Right Center Back


In [67]:
import pandas as pd

# Lista de (player_id, team) válidos
ids_y_equipos_validos = [
    (3083, "Tottenham Hotspur"),
    (3089, "Manchester City"),
    (3302, "Watford"),
    (5474, "Inter Milan"),
    (5687, "Eibar"),
    (6593, "Real Betis"),
    (6743, "Rayo Vallecano"),
    (6950, "Swansea City"),
    (7141, "Levante UD"),
    (7170, "Carpi"),
    (7788, "Torino"),
    (7852, "Genoa"),
    (8217, "Juventus"),
    (9461, "Norwich City"),
    (10881, "Sevilla"),
    (17520, "VfB Stuttgart"),
    (18404, "Rayo Vallecano"),
    (23488, "Watford"),
    (26211, "Levante UD"),
    (27341, "Celta Vigo"),
    (27853, "Fiorentina"),
]

# Convertir a DataFrame
df_validos = pd.DataFrame(ids_y_equipos_validos, columns=['player_id', 'team'])

# Filtrar duplicados por player_id
duplicados = jugadores_total[jugadores_total.duplicated(subset='player_id', keep=False)]

# Mantener solo los duplicados correctos (player_id + team que coincidan con la lista)
duplicados_validos = duplicados.merge(df_validos, on=['player_id', 'team'])

# Mantener los jugadores únicos que no están duplicados
no_duplicados = jugadores_total[~jugadores_total['player_id'].isin(duplicados['player_id'])]

# Unir los válidos con los no duplicados
jugadores_total = pd.concat([no_duplicados, duplicados_validos], ignore_index=True)

# Ver resultado



In [68]:
# Filtrar jugadores que están cedidos (club_loaned_from no es NaN)
cedidos = jugadores_total[jugadores_total['club_loaned_from'].notna()]

# Mostrar algunas columnas relevantes
print(cedidos[['player_id', 'player_name', 'team', 'club_loaned_from']])


      player_id                    player_name                    team  \
10         3379                Alphonse Areola              Villarreal   
23         4418           Matheus Dória Macedo                 Granada   
65         5678   Jefferson Andrés Lerma Solís              Levante UD   
68         5719        Marco Asensio Willemsen                Espanyol   
70         5724                    Fayçal Fajr  RC Deportivo La Coruña   
...         ...                            ...                     ...   
1738      10902                Philipp Hosiner                 FC Köln   
1782      16502            László Kleinheisler           Werder Bremen   
1889       6743  Manuel Rolando Iturra Urrutia          Rayo Vallecano   
1890       7141                 Giuseppe Rossi              Levante UD   
1899       9461                     Timm Klose            Norwich City   

            club_loaned_from  
10       Paris Saint-Germain  
23    Olympique de Marseille  
65            Atlé

In [69]:
jugadores_total["team"].unique()

array(['Real Sociedad', 'Real Madrid', 'Málaga', 'Sevilla', 'Granada',
       'Valencia', 'Villarreal', 'Barcelona', 'Real Betis', 'Las Palmas',
       'Athletic Club', 'Rayo Vallecano', 'Espanyol', 'Atlético Madrid',
       'Celta Vigo', 'RC Deportivo La Coruña', 'Levante UD', 'Eibar',
       'Sporting Gijón', 'Getafe', 'Chelsea', 'West Bromwich Albion',
       'Manchester United', 'Tottenham Hotspur', 'Aston Villa',
       'Newcastle United', 'AFC Bournemouth', 'Manchester City',
       'West Ham United', 'Crystal Palace', 'Southampton', 'Sunderland',
       'Everton', 'Swansea City', 'Watford', 'Stoke City',
       'Leicester City', 'Norwich City', 'Liverpool', 'Arsenal', 'Torino',
       'Lazio', 'Fiorentina', 'AS Roma', 'AC Milan', 'Empoli', 'Genoa',
       'Inter Milan', 'Juventus', 'Atalanta', 'Udinese', 'Napoli',
       'Sassuolo', 'Hellas Verona', 'Palermo', 'Sampdoria', 'Bologna',
       'Chievo', 'Carpi', 'Frosinone', 'Ingolstadt', 'Hannover 96',
       'Schalke 04', 'Bayer 

In [70]:
jugadores_total["club_loaned_from"].unique()

array([nan, 'Paris Saint-Germain', 'Olympique de Marseille',
       'Atlético Huila', 'Real Madrid CF', 'Elche CF', 'AC Milan',
       'SC Braga', 'Atlético de Madrid', 'Rubin Kazan', 'Granada CF',
       'Derby County', 'Valencia CF', 'FC Porto', 'Watford', 'SL Benfica',
       'Roma', 'FC Barcelona', 'Inter', 'Liverpool', 'Galatasaray SK',
       'SV Werder Bremen', 'Villarreal CF', "CD O'Higgins", 'Sporting CP',
       'Celtic', 'Udinese Calcio', 'Cruz Azul', 'Norwich City',
       'Sunderland', 'Swansea City', 'AS Monaco', 'Tottenham Hotspur',
       'Chelsea', 'Stade Rennais FC', 'Fulham', 'Arsenal',
       'West Ham United', 'VfB Stuttgart', 'Zenit St. Petersburg',
       'Borussia Dortmund', 'U.S. Sassuolo Calcio', 'Palermo', 'Pescara',
       'Napoli', 'U.C. Sampdoria', 'Genoa', 'Juventus', 'Cagliari',
       'Grasshopper Club Zürich', 'FC Sion', 'Fenerbahçe SK',
       'Hellas Verona', 'Standard de Liège', 'Club Brugge KV',
       'Sevilla FC', 'Bologna', 'FC Bayern München',


In [71]:
import numpy as np

# Función para truncar a 2 decimales (sin redondear) y formatear como string con dos decimales
def truncar_y_formatear(x):
    return f"{np.floor(x * 100) / 100:.2f}"

# Detectar columnas numéricas con decimales (float)
columnas_decimales = jugadores_total.select_dtypes(include='float').columns

# Aplicar el truncado y formateo
for col in columnas_decimales:
    jugadores_total[col] = jugadores_total[col].apply(truncar_y_formatear)

# Ver resultado en las columnas modificadas
print(jugadores_total[columnas_decimales].head())


     value_eur  wage_eur club_jersey_number club_contract_valid_until  \
0   1800000.00  20000.00              19.00                   2016.00   
1  16500000.00  90000.00              23.00                   2021.00   
2    375000.00   4000.00              27.00                   2016.00   
3   6500000.00  60000.00              12.00                   2019.00   
4   4500000.00  35000.00              21.00                   2019.00   

  release_clause_eur tackle_success_rate interception_success_rate  \
0                nan               56.86                     62.75   
1                nan               64.18                     61.11   
2                nan               66.67                     66.67   
3                nan              100.00                      0.00   
4                nan               64.70                     71.05   

  head_clearance_success_rate headed_shot_duel_rate pass_completion_rate  ...  \
0                       51.43                  0.00        

In [72]:
#jugadores_total.to_csv("datos/datos_jugadores_v3/jugadores_total.csv")

In [ ]:
# Filtrar solo defensas centrales (ajusta si es otro valor como 'Centre Back' o 'CB')
porteros = df[df['position'] == 'Goalkeeper'].copy()

# Seleccionar solo las columnas numéricas que representen métricas relevantes
# Aquí debes ajustar a tus nombres reales de columnas:
metricas = [
    'interceptions', 'tackles', 'clearances', 'aerial_duels_won',
    'passes_completed', 'pass_accuracy', 'blocks'
]

X = porteros[metricas].dropna()
